<a href="https://colab.research.google.com/github/aidantze/pesta-la-vista/blob/ml_models/ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Models
9517 Group Assignment

Constants

In [1]:
# KNN constants
RESOLUTION = 64                  # default 512
K_NEIGHBORS = 5                   # default 5

# Noise and filter constants
APPLY_CORRUPTION = False
CORRUPT_TYPE = 'gaussian_noise'   # one of: 'gaussian_noise', 'salt_pepper_noise', 'gaussian_blur'
CORRUPT_STRENGTH = 0.05           # e.g., 0.05 = 5% noise or 5x5 kernel blur

Setup

In [3]:
!pip install -q kagglehub pyyaml

import numpy as np
import cv2
import kagglehub
import pathlib
import yaml
import shutil
import os
import time

from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
import torch

# Check for T4 availability on Colab (DON'T CHANGE)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected. Go to Runtime -> Change runtime type ->  GPU")

# Kaggle Download via CLI API (see their website - DON'T CHANGE)
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")

# Saved the images to a local path to increase efficiency
local_path = pathlib.Path("/content/datasets/crop-pests")
local_path.parent.mkdir(parents=True, exist_ok=True)
shutil.copytree(path, local_path, dirs_exist_ok=True)

# YAML CONGIF (DON'T CHANGE)
data_yaml_path = local_path / "data.yaml"
data_cfg = {
    "path": str(local_path),
    "train": "train/images",
    "val":   "valid/images",
    "test":  "test/images",
    "names": [
        "ant", "bee", "beetle", "caterpillar", "earthworm", "earwig",
        "grasshopper", "moth", "slug", "snail", "wasp", "weevil"
    ]
}

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f)


CUDA available: False
No GPU detected. Go to Runtime -> Change runtime type ->  GPU
Using Colab cache for faster access to the 'crop-pests-dataset' dataset.


In [4]:
# Data Loading Function for KNN
CLASS_NAMES = data_cfg["names"]
CLASS_MAP = {name: i for i, name in enumerate(CLASS_NAMES)}

# def load_split_data(base_path, split_name, class_map, img_size):
#     """Loads and preprocesses images and labels for a specific data split."""
#     X_list = []
#     y_list = []

#     # We must traverse the actual class folders within the split (e.g., train/ant, train/bee)
#     split_dir = base_path / split_name

#     for class_name, label_id in class_map.items():
#         image_folder = split_dir / "images" / class_name
#         print(image_folder)

#         if image_folder.exists():
#             for img_file in image_folder.glob("*.jpg"):
#                 img = cv2.imread(str(img_file), cv2.IMREAD_GRAYSCALE)

#                 if img is not None:
#                     # Resize, flatten, and append
#                     img = cv2.resize(img, (img_size, img_size))
#                     img_normalized = img.astype('float32') / 255.0

#                     X_list.append(img_normalized.flatten())
#                     y_list.append(label_id)

#     X = np.array(X_list, dtype='float32')
#     y = np.array(y_list)

#     # Normalize pixel values (0-255 -> 0.0-1.0)
#     # X /= 255.0

#     # print(f"Loaded {len(X)} samples for {split_name}.")
#     # return X, y

#     if X.ndim == 1 and len(X) > 0:
#         # This handles the rare case where np.array(X_list) fails to stack
#         # (e.g., if X_list contains varying types). We reshape explicitly.
#         # This requires all elements in X_list to have the same length.
#         try:
#             target_features = img_size * img_size * 3
#             X = X.reshape(-1, target_features)
#             print("Successfully reshaped 1D array to 2D.")
#         except ValueError as e:
#             print(f"Error during final reshape check: {e}")
#             # The issue might be inconsistent feature counts.
#             return np.array([]), np.array([])

#     print(f"Loaded {len(X)} samples for {split_name}. Final shape: {X.shape}")
#     return X, y

def load_split_data(base_path, split_name, class_map, img_size):
    """
    Loads images and extracts the majority class label from the YOLO-style
    label files for KNN classification.
    """
    X_list = []
    y_list = []

    split_dir = base_path / split_name
    image_dir = split_dir / "images"
    label_dir = split_dir / "labels"

    print(f"\nProcessing images in: {image_dir}")

    if not image_dir.exists() or not label_dir.exists():
        print(f"Error: Could not find 'images' or 'labels' in {split_dir}")
        return np.array([]), np.array([])

    for img_file in image_dir.glob("*.jpg"):

        # 1. Determine the corresponding label file name (.txt)
        label_file = label_dir / (img_file.stem + ".txt")

        # 2. Check and process label file
        if not label_file.exists():
            # print(f"Warning: Missing label file for {img_file.name}") # Too verbose
            continue

        try:
            # Read all lines from the label file
            with open(label_file, 'r') as f:
                lines = f.readlines()

            if not lines:
                continue # Skip if label file is empty

            # Extract all class IDs (the first number in each row)
            class_ids = [int(line.strip().split()[0]) for line in lines]

            # Find the majority class ID (the most frequent insect)
            # bincount is efficient for finding frequencies of non-negative integers
            class_counts = np.bincount(class_ids)
            majority_class_id = np.argmax(class_counts)

        except Exception as e:
            print(f"Error reading label {label_file.name}: {e}")
            continue

        # 3. Load and Preprocess Image
        # Load as color image (3 channels) for consistent feature count
        img = cv2.imread(str(img_file), cv2.IMREAD_COLOR)

        if img is not None:
            # Resize, flatten, and normalize
            img = cv2.resize(img, (img_size, img_size))
            img_normalized = img.astype('float32') / 255.0

            X_list.append(img_normalized.flatten())
            y_list.append(majority_class_id) # Use the derived majority class

    # 4. Final array creation (handles the 2D shape requirement)
    X = np.array(X_list, dtype='float32')
    y = np.array(y_list)

    print(f"Loaded {len(X)} samples for {split_name}. Final shape: {X.shape}")
    return X, y


Noise and Filter Analysis

In [5]:
def apply_corruption_to_folder(source_dir, destination_dir, corruption_type, strength=0.01):
    """
    Copies images from source to destination and applies a specified corruption.
    (Function body is omitted here for brevity, assuming the user's provided code is used)
    """
    if destination_dir.exists():
        shutil.rmtree(destination_dir)
    shutil.copytree(source_dir, destination_dir)

    if corruption_type == 'gaussian_noise':
        sigma = int(strength * 255)
        print(f"\nApplying {corruption_type} (Sigma: {sigma}) to images in {destination_dir.name}...")
        for img_file in destination_dir.glob('*.jpg'):
            img = cv2.imread(str(img_file))
            if img is None: continue
            noise = np.random.normal(0, sigma, img.shape).astype('uint8')
            corrupted_img = cv2.add(img, noise)
            cv2.imwrite(str(img_file), corrupted_img)

    elif corruption_type == 'salt_pepper_noise':
        ratio = strength
        print(f"\nApplying {corruption_type} (Ratio: {ratio}) to images in {destination_dir.name}...")
        for img_file in destination_dir.glob('*.jpg'):
            img = cv2.imread(str(img_file))
            if img is None: continue
            corrupted_img = img.copy()
            total_pixels = img.size
            num_salt_pepper = int(ratio * total_pixels / img.shape[2])
            coords = [np.random.randint(0, i - 1, num_salt_pepper) for i in img.shape]
            corrupted_img[coords[0], coords[1], coords[2]] = 255
            coords = [np.random.randint(0, i - 1, num_salt_pepper) for i in img.shape]
            corrupted_img[coords[0], coords[1], coords[2]] = 0
            cv2.imwrite(str(img_file), corrupted_img)

    elif corruption_type == 'gaussian_blur':
        ksize = int(strength * 100) if int(strength * 100) % 2 != 0 else int(strength * 100) + 1
        print(f"\nApplying {corruption_type} (ksize: {ksize}) to images in {destination_dir.name}...")
        for img_file in destination_dir.glob('*.jpg'):
            img = cv2.imread(str(img_file))
            if img is None: continue
            corrupted_img = cv2.GaussianBlur(img, (ksize, ksize), 0)
            cv2.imwrite(str(img_file), corrupted_img)

    print("Corruption application complete.")


VAL_DIR_NAME = "valid"
if APPLY_CORRUPTION:
    original_val_path = local_path / VAL_DIR_NAME
    corrupt_val_path_name = f"{VAL_DIR_NAME}_corrupted"
    corrupt_val_path = local_path / corrupt_val_path_name

    # Create the corrupted validation set by copying and modifying the 'valid' split
    apply_corruption_to_folder(
        original_val_path,
        corrupt_val_path,
        CORRUPT_TYPE,
        CORRUPT_STRENGTH
    )
    VAL_DIR_NAME = corrupt_val_path_name # Switch the validation directory name

Load Data Splits

In [6]:
# --- 4. Load Data Splits ---
X_train, y_train = load_split_data(local_path, "train", CLASS_MAP, RESOLUTION)

# Load validation data (clean or corrupted)
X_val, y_val = load_split_data(local_path, VAL_DIR_NAME, CLASS_MAP, RESOLUTION)

# Load test data
X_test, y_test = load_split_data(local_path, "test", CLASS_MAP, RESOLUTION)


Processing images in: /content/datasets/crop-pests/train/images
Loaded 11499 samples for train. Final shape: (11499, 12288)

Processing images in: /content/datasets/crop-pests/valid/images
Loaded 1095 samples for valid. Final shape: (1095, 12288)

Processing images in: /content/datasets/crop-pests/test/images
Loaded 546 samples for test. Final shape: (546, 12288)


KNN Classifier — Train model, prediction and evaluation

In [10]:
# Training the Model
knn_classifier = KNeighborsClassifier(n_neighbors=K_NEIGHBORS)

start = time.time()
knn_classifier.fit(X_train, y_train)
end = time.time()
train_time = end - start

start_kernel = time.time()
knn_val_predictions = knn_classifier.predict(X_val)
end = time.time()
test_time = end - start

start = time.time()
knn_val_probas = knn_classifier.predict_proba(X_val)
end = time.time()
val_time = end - start

# knn_val_predictions = knn_classifier.predict(X_val)
# knn_val_probas = knn_classifier.predict_proba(X_val)

# def evaluate_knn(X, y, dataset_name, classifier):
#     """Predicts and prints metrics for a given dataset split."""
#     start_time = time.time()
#     predictions = classifier.predict(X)
#     runtime = time.time() - start_time

#     print(f"\n=== KNN ({dataset_name}) ===")
#     print(f"Runtime: {runtime:.2f} seconds")
#     print("Accuracy:", metrics.accuracy_score(y, predictions))
#     print("\nClassification Report:")
#     print(metrics.classification_report(y, predictions, target_names=CLASS_NAMES))

#     return runtime

# # Validate the Model
# val_time = evaluate_knn(X_val, y_val, "Validation Data", knn_classifier)

# # Predict/Test the Model
# test_time = evaluate_knn(X_test, y_test, "Test Data", knn_classifier)


In [11]:
# evaluation metrics
# We use the 'weighted' average suitable for multi-class, potentially imbalanced data.
accuracy = metrics.accuracy_score(y_val, knn_val_predictions)
precision, recall, f1_score, _ = metrics.precision_recall_fscore_support(
    y_val, knn_val_predictions, average='weighted', zero_division=0
)

# AUC is calculated in a One-vs-Rest fashion for multi-class problems.
# The true labels (y_val) must be binarized first.
lb = LabelBinarizer()
y_val_binarized = lb.fit_transform(y_val)

# We use the 'weighted' average to account for class support
auc_score = metrics.roc_auc_score(
    y_val_binarized,
    knn_val_probas,
    average='weighted',
    multi_class='ovr'
)

def format_time(seconds):
    """Converts total seconds into minutes and seconds format."""
    # Ensure all time variables (train_time, val_time, test_time) are available
    mins, secs = divmod(seconds, 60)
    return f"{int(mins):0d}m {secs:.2f}s"


# --- 4. Output Refactored Metrics ---
print(f"Results for KNN (K={K_NEIGHBORS}) with image size {RESOLUTION} and {f"{CORRUPT_TYPE}, strength set to {CORRUPT_STRENGTH}" if APPLY_CORRUPTION else "no filters"}:\n")

# NOTE: mAP is not available for KNN (Classification)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (Weighted): {precision:.4f}")
print(f"Recall (Weighted): {recall:.4f}")
print(f"F1-Score (Weighted): {f1_score:.4f}")
print(f"Area Under the Curve (AUC): {auc_score:.4f}")

print(f"\nTraining Time (Total): {format_time(train_time)}")
print(f"Validation Time (Prediction): {format_time(val_time)}")

Results for KNN (K=5) with image size 64 and no filters:

Accuracy: 0.1653
Precision (Weighted): 0.1874
Recall (Weighted): 0.1653
F1-Score (Weighted): 0.1505
Area Under the Curve (AUC): 0.5918

Training Time (Total): 0m 0.12s
Validation Time (Prediction): 0m 12.28s
